In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Load the trained model
model = tf.keras.models.load_model('handover_prediction_model')

# Load the data
mobility_data = pd.read_parquet('mobility_data.parquet')

In [ ]:
def prepare_sequence_for_prediction(user_data, sequence_length=20):
    """Prepare a single sequence for prediction"""
    features = [
        'x', 'y', 'velocity', 'heading', 'signal_strength', 
        'sinr', 'network_load', 'throughput_mbps'
    ]
    
    # Add time-based features
    user_data['hour_sin'] = np.sin(2 * np.pi * user_data['timestamp'].dt.hour / 24)
    user_data['hour_cos'] = np.cos(2 * np.pi * user_data['timestamp'].dt.hour / 24)
    user_data['day_of_week_sin'] = np.sin(2 * np.pi * user_data['timestamp'].dt.dayofweek / 7)
    user_data['day_of_week_cos'] = np.cos(2 * np.pi * user_data['timestamp'].dt.dayofweek / 7)
    
    features += ['hour_sin', 'hour_cos', 'day_of_week_sin', 'day_of_week_cos']
    
    # One-hot encode categorical features
    if 'pattern_type' in user_data.columns:
        pattern_dummies = pd.get_dummies(user_data['pattern_type'], prefix='pattern')
        user_data = pd.concat([user_data, pattern_dummies], axis=1)
        features += list(pattern_dummies.columns)
    
    if 'device_type' in user_data.columns:
        device_dummies = pd.get_dummies(user_data['device_type'], prefix='device')
        user_data = pd.concat([user_data, device_dummies], axis=1)
        features += list(device_dummies.columns)
    
    # Get the last sequence_length rows
    sequence = user_data.iloc[-sequence_length:][features].values
    
    return sequence.reshape(1, sequence_length, -1)

In [ ]:
# Select a random user for demonstration
user_id = np.random.choice(mobility_data['user_id'].unique())
user_data = mobility_data[mobility_data['user_id'] == user_id].sort_values('timestamp')

# Prepare sequence for prediction
sequence = prepare_sequence_for_prediction(user_data)

# Make prediction
prediction = model.predict(sequence)[0][0]
print(f"Predicted probability of handover needed: {prediction:.4f}")
print(f"Prediction threshold (0.5): {'Handover needed' if prediction > 0.5 else 'No handover needed'}")

In [ ]:
# Visualize user trajectory and predictions
plt.figure(figsize=(12, 8))

# Plot user trajectory
plt.scatter(user_data['x'], user_data['y'], c='blue', alpha=0.5, label='User trajectory')

# Plot cell towers
grid_size = 10
for i in range(grid_size):
    for j in range(grid_size):
        plt.scatter(i*1000, j*1000, c='red', marker='^', s=100, alpha=0.3)

# Highlight current position
current_pos = user_data.iloc[-1]
plt.scatter(current_pos['x'], current_pos['y'], c='green', s=200, label='Current position')

plt.title(f'User Trajectory and Handover Prediction\nPrediction: {prediction:.4f}')
plt.xlabel('X Position (m)')
plt.ylabel('Y Position (m)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot signal strength and network metrics over time
plt.figure(figsize=(15, 10))

plt.subplot(311)
plt.plot(user_data['timestamp'], user_data['signal_strength'], label='Signal Strength')
plt.title('Signal Strength Over Time')
plt.xlabel('Time')
plt.ylabel('Signal Strength (dBm)')
plt.legend()

plt.subplot(312)
plt.plot(user_data['timestamp'], user_data['network_load'], label='Network Load')
plt.title('Network Load Over Time')
plt.xlabel('Time')
plt.ylabel('Network Load')
plt.legend()

plt.subplot(313)
plt.plot(user_data['timestamp'], user_data['sinr'], label='SINR')
plt.title('SINR Over Time')
plt.xlabel('Time')
plt.ylabel('SINR (dB)')
plt.legend()

plt.tight_layout()
plt.show()